<a href="https://colab.research.google.com/github/vickytoriag/Home-Work/blob/main/%D0%94%D0%97_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Задание**
---


Ваша задача — реализовать класс Account, который моделирует поведение банковского счёта. Этот класс должен не только выполнять базовые операции,но и вести детальный учёт всех действий, а также предоставлять  аналитику по истории операций.

Этап 1. Реализация базового класса Account
Класс должен быть инициализирован с параметрами:
* account_holder (str): имя владельца счёта;
* balance (float, по умолчанию 0): начальный баланс счёта, не может быть
отрицательным.

Атрибуты:
* holder: хранит имя владельца;
* _balance: приватный атрибут для хранения текущего баланса;
* operations_history: список или другая структура для хранения истории
операций.

Важно: каждая операция должна храниться не просто как число, а как структурированная информация, например, словарь или кортеж. Минимальный набор данных для операции: тип операции ('deposit' или 'withdraw'), сумма, дата и время операции, текущий баланс после операции, статус ('success' или 'fail').

Этап 2. Реалирзация методов

1. __init__(self, account_holder, balance=0): конструктор;
2. deposit(self, amount): метод для пополнения счёта:
  * принимает сумму (должна быть положительной);
  * в случае успеха обновляет баланс и добавляет запись в историю операций.
3. withdraw(self, amount): метод для снятия средств:
* принимает сумму (должна быть положительной);
* проверяет, достаточно ли средств на счёте, если нет — операция не проходит, но ее попытка с статусом 'fail' все равно фиксируется в истории;
* в случае успеха обновляет баланс и добавляет запись.
4. get_balance(self): метод, который возвращает текущий баланс
5. get_history(self): метод, который возвращает историю операций.

Важно: продумайте, в каком формате его вернуть. Для работы с датой и временем
используйте модуль datetime. Получить текущее время можно с помощью
datetime.now().

Этап 3. Реализация наследования.
Создайте класс CreditAccount(Account), который наследует всю функциональность класса Account и добавляет новую.

Особенности кредитного счёта:
1. При инициализации принимает дополнительный параметр credit_limit
(кредитный лимит). Баланс такого счета может быть отрицательным, но не ниже значения -credit_limit.
2. По запросу показывает, сколько кредитных средств еще доступно (текущий баланс + кредитный лимит).
3. В историю операций добавляется информацию о том, были ли использованы кредитные средства в данной операции.

In [ ]:
from datetime import datetime


# ===== Account =====
# Этот класс моделирует банковский счет:
# хранит владельца, баланс и историю операций.
class Account:
    def __init__(self, account_holder, balance=0):
        # Создаем счет и проверяем, что
        # стартовый баланс не отрицательный
        self._validate_initial_balance(balance)
        self.holder = account_holder
        self._balance = float(balance)
        self.operations_history = []

    def _validate_initial_balance(self, balance):
        # Проверка стартового баланса
        if balance < 0:
            raise ValueError("Начальный баланс не может быть отрицательным")

    def deposit(self, amount):
        # Пополнение: сумма должна быть положительной
        ok = amount > 0
        if ok:
            self._balance += amount
        # Записываем операцию в историю, даже если
        # пополнение не прошло
        self._add_operation("deposit", amount, ok)
        return ok

    def withdraw(self, amount):
        # Снятие: сумма должна быть положительной и
        # не больше текущего баланса
        ok = amount > 0 and amount <= self._balance
        if ok:
            self._balance -= amount
        # Записываем операцию в историю, даже если снятие не прошло
        self._add_operation("withdraw", amount, ok)
        return ok

    def get_balance(self):
        # Возвращаем текущий баланс
        return self._balance

    def get_history(self):
        # Возвращаем историю операций
        return self.operations_history

    def stats(self):
        # Минимальная аналитика по операциям: сколько всего,
        # сколько успешных и неуспешных
        total = len(self.operations_history)
        success = sum(1 for op in self.operations_history if op["status"] == "success")
        return {"total": total, "success": success, "fail": total - success}

    def print_history(self):
        # История операций
        for op in self.operations_history:
            print(
                op["datetime"].strftime("%d.%m.%Y %H:%M"),
                op["type"],
                op["amount"],
                op["status"],
                op["balance_after"],
                op.get("credit_used", "")
            )

    def _add_operation(self, op_type, amount, ok, credit_used=None):
        # Формируем запись об операции и
        # добавляем ее в историю
        record = {
            "type": op_type,
            "amount": float(amount),
            "datetime": datetime.now(),
            "balance_after": self._balance,
            "status": "success" if ok else "fail"
        }
        # Для кредитного счета дополнительно
        # записываем, использовался ли кредит
        if credit_used is not None:
            record["credit_used"] = credit_used

        self.operations_history.append(record)


# ===== CreditAccount =====
# Кредитный счет: можно уходить в минус,
# но не ниже -credit_limit. Наследует весь
# функционал Account, но добавляет кредитный
# лимит и логику снятия.
class CreditAccount(Account):
    def __init__(self, account_holder, balance=0, credit_limit=0):
        # Проверяем кредитный лимит
        if credit_limit < 0:
            raise ValueError("Кредитный лимит не может быть отрицательным")

        # Сохраняем лимит до вызова super(), потому
        # что он нужен в проверке баланса
        self.credit_limit = float(credit_limit)

        # Создаем счет через базовый класс
        super().__init__(account_holder, balance)

    def _validate_initial_balance(self, balance):
        # Для кредитного счета стартовый баланс
        # может быть отрицательным, но не ниже
        # -credit_limit
        if balance < -self.credit_limit:
            raise ValueError("Баланс меньше минус кредитного лимита")

    def deposit(self, amount):
        # Пополнение кредитного счета: сумма
        # должна быть положительной
        ok = amount > 0
        if ok:
            self._balance += amount
        # Для пополнения кредит обычно не
        # "используется", поэтому credit_used=False
        self._add_operation("deposit", amount, ok, credit_used=False)
        return ok

    def withdraw(self, amount):
        # Снятие с учетом кредитного лимита
        if amount <= 0:
            # Неверная сумма: операция не проходит,
            # но записывается в историю
            self._add_operation("withdraw", amount, False, credit_used=False)
            return False

        new_balance = self._balance - amount

        # Кредит считается использованным, если
        # после операции баланс уходит в минус
        credit_used = new_balance >= -self.credit_limit and new_balance < 0

        # Если лимит не превышен, меняем баланс
        if new_balance >= -self.credit_limit:
            self._balance = new_balance

        # Записываем операцию в историю
        self._add_operation("withdraw", amount, new_balance >= -self.credit_limit, credit_used=credit_used)
        return new_balance >= -self.credit_limit

    def available_credit(self):
        # Сколько кредитных средств еще доступно:
        # текущий баланс + лимит
        return self._balance + self.credit_limit


# ===== ПРОВЕРКА =====

print("Account 1")
account_1 = Account("Account 1", 100)
account_1.deposit(50)      # успешно
account_1.withdraw(200)    # неуспешно (не хватает денег)
print("Balance:", account_1.get_balance())
account_1.print_history()
print("Stats:", account_1.stats())

print()

print("Account 2")
account_2 = CreditAccount("Account 2", -50, 300)
account_2.withdraw(200)    # успешно, уходит глубже в минус, но в пределах лимита
account_2.deposit(100)     # успешно
print("Balance:", account_2.get_balance())
print("Available credit:", account_2.available_credit())
account_2.print_history()
print("Stats:", account_2.stats())



---


* В работе реализован класс Account, в котором можно пополнять счёт, снимать средства и сохраняется история операций с датой и статусом.

* Также реализован класс CreditAccount на его основе, который позволяет использовать кредитные средства в пределах заданного лимита.

* Все операции, включая неуспешные, сохраняются в истории. Реализована простая аналитика по операциям.

* Внизу кода приведена проверка корректности работы классов.


---

